In [1]:
import msgpack
import random
import tensorflow as tf
from tensorflow import keras
import numpy as np
import json

from scenes import *
import pyngp as ngp

2024-02-05 20:48:32.803112: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-05 20:48:33.411706: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# !python ../scripts/run.py --scene=../BananaScene --save_screenshot="../SaveModel.msgpack" --n_steps=10000

In [3]:
# data_loaded = None
# parameters = None
# with open("../SavedModel.msgpack", "rb") as data_file:
#     byte_data = data_file.read()
#     data_loaded = msgpack.unpackb(byte_data)
#     print(data_loaded['snapshot']['render_aabb'])
#     parameters = np.frombuffer(data_loaded["snapshot"]["params_binary"], dtype=np.float16).copy()
#     print(len(parameters.tobytes()))
#     # for i in range(100):
#     #     # if i > 20:
#     #     #     break
#     #     # print(test[i])
#     #     randNum = random.randint(-5, 5)
#     #     test[i] += randNum
#     # print(len(test))
#     # print(data_loaded['snapshot']['n_params'])
#     # print(test.max())
#     # print(test.min())
#     # data_loaded["snapshot"]["params_binary"] = test.tobytes()
#     # with open("../Test2.msgpack", "wb") as file:
#     #     file.write(msgpack.packb(data_loaded))

In [4]:
# array = None
# data_loaded = None
# with open("../SavedModel.msgpack", "rb") as data_file:
#     byte_data = data_file.read()
#     data_loaded = msgpack.unpackb(byte_data)
#     array = bytearray(data_loaded['snapshot']['params_binary'])
#     # TODO: So we're modifying the full feature grid?
#     for i in range(10000):
#         randNum = random.randint(1, 5)
#         if array[i] + randNum < 256:
#             array[i] += randNum
    
#     data_loaded['snapshot']['params_binary'] = bytes(array)

#     with open("../Test.msgpack", "wb") as file:
#         file.write(msgpack.packb(data_loaded))

In [5]:
def get_scene(scene):
	for scenes in [scenes_sdf, scenes_nerf, scenes_image, scenes_volume]:
		if scene in scenes:
			return scenes[scene]
	return None

def Train_NeRF(testbed, load_snapshot, screenshot_transforms, screenshot_dir, sharpen=0, exposure=0.0, screenshot_frames=None, screenshot_spp=16, width=224, height=224, network=None):

	if load_snapshot:
		scene_info = get_scene(load_snapshot)
		if scene_info is not None:
			load_snapshot = default_snapshot_filename(scene_info)
		testbed.load_snapshot(load_snapshot)

	ref_transforms = {}
	if screenshot_transforms: # try to load the given file straight away
		print("Screenshot transforms from ", screenshot_transforms)
		with open(screenshot_transforms) as f:
			ref_transforms = json.load(f)

	if testbed.mode == ngp.TestbedMode.Sdf:
		testbed.tonemap_curve = ngp.TonemapCurve.ACES

	testbed.nerf.sharpen = float(sharpen)
	testbed.exposure = exposure
	testbed.shall_train = True


	testbed.nerf.render_with_lens_distortion = True

	if ref_transforms:
		testbed.fov_axis = 0
		testbed.fov = ref_transforms["camera_angle_x"] * 180 / np.pi
		if not screenshot_frames:
			screenshot_frames = range(len(ref_transforms["frames"]))

		for idx in screenshot_frames:
			f = ref_transforms["frames"][int(idx)]
			if 'transform_matrix' in f:
				cam_matrix = f['transform_matrix']
			elif 'transform_matrix_start' in f:
				cam_matrix = f["transform_matrix_start"]
			else:
				raise KeyError()
			testbed.set_nerf_camera_matrix(np.matrix(cam_matrix)[:-1,:])
			outname = os.path.join(screenshot_dir, os.path.basename(f["file_path"]))

			# Some NeRF datasets lack the .png suffix in the dataset metadata
			if not os.path.splitext(outname)[1]:
				outname = outname + ".png"

			image = testbed.render(width or int(ref_transforms["w"]), height or int(ref_transforms["h"]), screenshot_spp, True)
			os.makedirs(os.path.dirname(outname), exist_ok=True)
			write_image(outname, image)


In [6]:
def loadNeRFData(fileName = "../SavedModel.msgpack"):
    with open(fileName, "rb") as data_file:
        byte_data = data_file.read()
        data_loaded = msgpack.unpackb(byte_data)
        parameters = np.frombuffer(data_loaded["snapshot"]["params_binary"], dtype=np.float16).copy()
        return data_loaded, parameters
        
def SaveParameters(data_loaded, parameters, outputFileName = "../Test.msgpack"):
    data_loaded['snapshot']['params_binary'] = parameters.tobytes()

    with open(outputFileName, "wb") as file:
        file.write(msgpack.packb(data_loaded))

In [7]:
pretrained_model = tf.keras.applications.MobileNetV2()
pretrained_model.trainable = False

decode_predictions = tf.keras.applications.mobilenet_v2.decode_predictions

2024-02-05 20:48:34.543058: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:1000] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-02-05 20:48:34.543686: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:1000] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-02-05 20:48:34.588769: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2252] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required

In [13]:
# import gymnasium as gym
# from gymnasium import spaces
# import numpy as np
# from stable_baselines3 import PPO, A2C
# from stable_baselines3.common.env_checker import check_env
# from PIL import Image
# from skimage.metrics import peak_signal_noise_ratio


# # Preprocess numpy images
# def preprocess(image, unproc=False):
#     if not unproc:
#         image = tf.keras.applications.mobilenet_v2.preprocess_input(image)
#         return np.asarray(image, dtype=np.float32)
#     else:
#         return np.asarray(image, dtype=np.uint8)
#     # Iteration ste

# # Loads images from path
# def prep_im(image_path, unproc=False):
#     image = np.asarray(Image.open(image_path), dtype=np.float32)
#     # ## print(image_raw.shape())
#     image = image[None, ...]
#     image = preprocess(image, unproc=unproc)
#     #     image_probs = pretrained_model.predict(image)
#     #     plot_fig(image, image_probs)
#     return image

# pred = pretrained_model.predict(prep_im("../../Slug69Percent.jpg"))
# decode_predictions(pred, top=5)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step


[[('n01945685', 'slug', 0.6970356),
  ('n07753592', 'banana', 0.03878794),
  ('n02264363', 'lacewing', 0.028227882),
  ('n01930112', 'nematode', 0.026499515),
  ('n01728572', 'thunder_snake', 0.010387602)]]

In [9]:
testbed = ngp.Testbed()

20:48:35 SUCCESS  Initialized CUDA 11.8. Active GPU is #1: NVIDIA GeForce RTX 3090 [86]
20:48:35 SUCCESS  Detected auxiliary GPUs:
20:48:35 SUCCESS    #0: NVIDIA GeForce RTX 3090 [86]


In [10]:
mse = keras.losses.MeanSquaredError()

In [11]:
import gymnasium as gym
from gymnasium import spaces
import numpy as np
from stable_baselines3 import PPO, A2C
from stable_baselines3.common.env_checker import check_env
from PIL import Image
from skimage.metrics import peak_signal_noise_ratio

logPath = '../AdvOutput/rewards.log'
# Preprocess numpy images
def preprocess(image, unproc=False):
    if not unproc:
        image = tf.keras.applications.mobilenet_v2.preprocess_input(image)
        return np.asarray(image, dtype=np.float32)
    else:
        return np.asarray(image, dtype=np.uint8)
    # Iteration ste

# Loads images from path
def prep_im(image_path, unproc=False):
    image = np.asarray(Image.open(image_path), dtype=np.float32)
    # ## print(image_raw.shape())
    image = image[None, ...]
    image = preprocess(image, unproc=unproc)
    #     image_probs = pretrained_model.predict(image)
    #     plot_fig(image, image_probs)
    return image

class InstantNGPEnv(gym.Env):
    def __init__(self):
        super(InstantNGPEnv, self).__init__()
        # self.reward_range = (-1,1)
        # Load Instant-NGP and MobileNetV2 models
        # self.instant_ngp = ...  # Load Instant-NGP model
        self.data_loaded, self.feature_grid = loadNeRFData()
        self.ground_truth_img = None
        self.log = open(logPath, 'a')


        self.mobilenetv2 = pretrained_model  # Load MobileNetV2 model
        self.true_class_id = 954
        self.target = "slug" # Target tiger shark class
        self.noiseAmount = 0.15
        self.max_reward = -np.inf
        self.epochs = 0
        self.targeted_grad = None
        self.unrendered = None
        self.state = 38 + random.randint(-3,3)

        # Define action and observation spaces
        self.action_space = spaces.Box(low=-0.005, high=0.005, shape=(13205056,), dtype=np.float32)  # Define space of allowable feature grid modifications
        self.observation_space = spaces.Box(low=0, high=np.inf, shape=(13205056,), dtype=np.float32) # Define state representation (e.g., feature grid, image)


    def reset(self, seed=0):
        super().reset(seed=seed)
        # Reset feature grid or load new one
        self.mobilenetv2.trainable = False
        self.ground_truth_img = prep_im("../NormalOutput/0081.jpg")
        self.data_loaded, self.feature_grid = loadNeRFData()
        # self.log.close()
        return (self.feature_grid, {})
    
    def get_reward(self, originalImage, renderedImage, predicted, min_psnr=30, max_psnr=50, mse_weight = -2, psnr_weight = .5):

        returned_reward = 0
        imageLoss = mse(originalImage, renderedImage)
        psnr = (peak_signal_noise_ratio(originalImage, renderedImage, data_range=255) - min_psnr) / (max_psnr - min_psnr)
        if predicted[1] == self.target:
            returned_reward = predicted[2] * 10 + psnr * psnr_weight + imageLoss * mse_weight
        else:
            returned_reward = -100

        print("Psnr: " + str(psnr) + ", MSE: " + str(imageLoss) + " for class " + predicted[1])
        return returned_reward

    def step(self, action):
        # Modify feature grid based on action
        self.feature_grid +=  action[0] # * self.noiseAmount  # Apply action to feature grid

        # Render image from modified feature grid
        SaveParameters(self.data_loaded, self.feature_grid)
        Train_NeRF(testbed, "../Test.msgpack", "../BananaScene/transforms.json", "../AdvOutput/")
        image = prep_im("../AdvOutput/0081.jpg")

        # Classify image using MobileNetV2
        prediction = self.mobilenetv2.predict(image)

        print("Top prediction:  " + str(decode_predictions(prediction, top=1)))

        decoded_prediction = decode_predictions(prediction, top=1)[0][0]
        # Calculate reward based on adversarial success and image quality
        reward = self.get_reward(self.ground_truth_img, image, decoded_prediction)  # Implement reward function
        # reward += (1 - (np.squeeze(prediction)[self.true_class_id])) # TODO: Adjust reward so model gets to slug with high accuracy.
        reward -= np.squeeze(prediction)[self.true_class_id] * 10
        # print("Banana: " + str(decode_predictions(prediction)) + ", conf: " + str(np.squeeze(prediction)[self.true_class_id]))
        # Determine if episode is done
        done = False

        # TODO: The reason why it keeps predicting the same thing has to do with this "done" flag and its criteria. Find another criteria.
        # done = True if reward <= -100 else False  # Define termination criteria
        truncated, info = False, {}
        
        if self.epochs % 10 == 0 or reward == self.max_reward:
        # if self.epochs % 250 == 0:
            print('saving tape...')
            self.log.write(
                '\n'
                + str(self.epochs)
                + ', '
                + str(decode_predictions(prediction, top=1))
            )
        self.epochs += 1

        info = {
            "Target" : self.target,
            "Predicted" : decoded_prediction[1],
            "Confidence" : decoded_prediction[2],
            "Epochs" : self.epochs
        }

        return self.feature_grid, reward, done, truncated, info

    
    def render(self):
        pass

    def close(self):
        self.log.close()

# Create environment instance
env = InstantNGPEnv()

# Create RL agent
model = PPO('MlpPolicy', env, device='cuda')

# Train the agent
# Evaluate the trained agent
episode_reward = 0
obs, info = env.reset()
_states = None
# TODO: Once done testing, comment out line below
# model = model.learn(total_timesteps=2000)

for i in range(2000):
    action, _states = model.predict(obs, _states)
    obs, reward, done, truncated, info = env.step(action)
    episode_reward += reward
    print("Current episode reward: " + str(episode_reward))
    

    if info['Target'] == info['Predicted'] and info['Confidence'] >= .65:
        print("Episode reward:", episode_reward)
        break
    # model = model.learn(total_timesteps=1, log_interval=4)



20:49:38 INFO     Loading network snapshot from: ../Test.msgpack
Screenshot transforms from  ../BananaScene/transforms.json
20:49:38 INFO     GridEncoding:  Nmin=16 b=3.28134 F=4 T=2^19 L=8
20:49:38 INFO     Density model: 3--[HashGrid]-->32--[FullyFusedMLP(neurons=64,layers=3)]-->1
20:49:38 INFO     Color model:   3--[Composite]-->16+16--[FullyFusedMLP(neurons=64,layers=4)]-->3
20:49:38 INFO       total_encoding_params=13194816 total_network_params=10240
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 559ms/step
Top prediction:  [[('n01945685', 'slug', 0.32665408)]]
Psnr: 2.151578353557924, MSE: tf.Tensor(0.003235368, shape=(), dtype=float32) for class slug
saving tape...
Current episode reward: tf.Tensor(1.7614489, shape=(), dtype=float32)
20:50:04 INFO     Loading network snapshot from: ../Test.msgpack
Screenshot transforms from  ../BananaScene/transforms.json
20:50:04 INFO     GridEncoding:  Nmin=16 b=3.28134 F=4 T=2^19 L=8
20:50:04 INFO     Density model: 3--[HashGrid]-->32--[FullyFusedMLP(neurons=64

In [12]:
# !python adversarialAttack.py --load_snapshot=../SavedModel.msgpack --screenshot_transforms=../BananaScene/transforms.json --screenshot_dir=../NormalOutput --width=224 --height=224


# Load Pretrained ImageNetV2